# PointCept PointTransformer

Based on https://github.com/Pointcept/Pointcept


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load Model

In [3]:
%cd /content/gdrive/MyDrive/Developer/
!git clone https://github.com/Pointcept/Pointcept
%cd Pointcept

/content/gdrive/MyDrive/Developer
fatal: destination path 'Pointcept' already exists and is not an empty directory.
/content/gdrive/MyDrive/Developer/Pointcept


In [ ]:
!conda create -n pointcept python=3.8 -y
!conda activate pointcept
!conda install ninja -y
# Choose version you want here: https://pytorch.org/get-started/previous-versions/
!conda install pytorch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1 cudatoolkit=11.3 -c pytorch -y
!conda install h5py pyyaml -c anaconda -y
!conda install sharedarray tensorboard tensorboardx yapf addict einops scipy plyfile termcolor timm -c conda-forge -y
!conda install pytorch-cluster pytorch-scatter pytorch-sparse -c pyg -y
!pip install torch-geometric

# spconv (SparseUNet)
# refer https://github.com/traveller59/spconv
!pip install spconv-cu113

# PTv1 & PTv2 or precise eval
%cd libs/pointops
# usual
!python setup.py install
# docker & multi GPU arch
!TORCH_CUDA_ARCH_LIST="ARCH LIST" python  setup.py install
# e.g. 7.5: RTX 3000; 8.0: a100 More available in: https://developer.nvidia.com/cuda-gpus
!TORCH_CUDA_ARCH_LIST="7.5 8.0" python  setup.py install
%cd ../..

# Open3D (visualization, optional)
!pip install open3d

Streaming output truncated to the last 5000 lines.











 ... (more hidden) ...





lerc-4.0.0           | 275 KB    | : 100% 1.0/1 [00:00<00:00,  1.17it/s]                





lerc-4.0.0           | 275 KB    | : 100% 1.0/1 [00:00<00:00,  1.17it/s]














pillow-9.4.0         | 44.3 MB   | :  19% 0.18831820713087974/1 [00:00<00:02,  3.51s/it]









pytorch-1.12.1       | 1.20 GB   | :   1% 0.008149624427199437/1 [00:01<01:29, 90.13s/it]
















torchvision-0.13.1   | 28.3 MB   | :  29% 0.2944914242548915/1 [00:01<00:01,  2.34s/it]


















 ... (more hidden) ...














pillow-9.4.0         | 44.3 MB   | :  22% 0.22217316571620643/1 [00:01<00:02,  3.41s/it]









pytorch-1.12.1       | 1.20 GB   | :   1% 0.009361227432808117/1 [00:01<01:30, 91.76s/it]
















torchvision-0.13.1   | 28.3 MB   | :  35% 0.3536107158032468/1 [00:01<00:01,  2.16s/it]


















 ... (more hidden) ...














pillow-9.4.0         | 44.3 MB   | 

/content/gdrive/MyDrive/Developer/Pointcept/libs/pointops
writing requirements to pointops.egg-info/requires.txt
writing top-level names to pointops.egg-info/top_level.txt
reading manifest file 'pointops.egg-info/SOURCES.txt'
writing manifest file 'pointops.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/Developer/Pointcept/libs/pointops/setup.py", line 19, in <module>
    setup(
  File "/usr/local/lib/python3.10/site-packages/setuptools/__init__.py", line 87, in setup
    return distutils.core.setup(**attrs)
  File "/usr/local/lib/python3.10/site-packages/setuptools/_distutils/core.py", line 185, in setup
    return run_commands(dist)
  File "/usr/local/lib/python3.10/site-packages/setuptools/_distutils/core.py", line 201, in run_commands
    dist.run_commands()
  File "/usr/local/lib/python3.10/site-packages/setuptools/_distutils/dist

In [ ]:
%cd /content/gdrive/MyDrive/Developer/Pointcept
from pointcept.models.point_transformer import PointTransformerSeg26
# from pointcept.models.point_transformer_v3 import PointTransformerV3

model = PointTransformerSeg26()

## Load Dataset

In [ ]:
%cd /content/gdrive/MyDrive/Developer/coarse-to-fine-shapes
from datasets.shapenet_data_pc import ShapeNet15kPointClouds
shapenet_location = "/content/gdrive/MyDrive/Developer/ShapeNetCore_pointclouds/ShapeNetCore.v2.PC15k"
dataset = ShapeNet15kPointClouds(root_dir=shapenet_location, tr_sample_size=1024, te_sample_size=1024)

## Overfit model on single pointcloud

In [ ]:
%cd /content/gdrive/MyDrive/Developer/coarse-to-fine-shapes

from metrics.chamfer_distance import np_chamfer_distance

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch

loss_fn = np_chamfer_distance

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
model.to(device)

for epoch in range(10):
  model.train(True)

  input = torch.unsqueeze(dataset[0]['train_points'], dim=0).to(device)
  output = torch.unsqueeze(dataset[0]['train_points'], dim=0).to(device)
  optimizer.zero_grad()
